In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as  px
import pandas as pd
import numpy as np

In [3]:
education = pd.read_csv("states_all.csv").assign(
    expenditure_per_student = lambda x: x["TOTAL_EXPENDITURE"] / x["GRADES_ALL_G"],
    above_avg_math8 = lambda x: np.where(x["AVG_MATH_8_SCORE"] > 278.6,'Above Avg', 'Below Avg')
)

In [ ]:
# Dropdown example
app= dash.Dash(__name__)

app.layout=html.Div([
    dcc.Dropdown(
        id="filter",
        options=education["STATE"].unique(),
        value=["CALIFORNIA"],
        multi=True
    ),
    dcc.Graph(id="graph")
])

@app.callback(Output('graph','figure'),
              Input('filter','value'))

def  plot_rev_scatter(state):
    fig=px.line(
        (
            education.query("STATE in @state and 1992 < YEAR < 2016")
            .groupby(["YEAR","STATE"])
            .sum()
            .reset_index()
        ),
        x='YEAR',
        y='TOTAL_EXPENDITURE',
        color='STATE',
        title=f"Expenditure Over Time, Selected States",
    )
    return fig

if __name__=="__main__":
    app.run_server(debug=True)

In [ ]:
# Checklist example
app= dash.Dash(__name__)
app.layout=html.Div([
    dcc.Checklist(
        id="filter",
        #options=['CALIFORNIA','TEXAS','FLORIDA','OREGON'],
        options=[{'label':state.title(), 'value':state} for state in education["STATE"].unique()],
        value=['CALIFORNIA','TEXAS']
    ),
    dcc.Graph(id="graph")
])

@app.callback(Output('graph','figure'),
              Input('filter','value'))

def  plot_rev_scatter(state):
    fig=px.line(
        (
            education.query("STATE in @state and 1992 < YEAR < 2016")
            .groupby(["YEAR","STATE"])
            .sum()
            .reset_index()
        ),
        x='YEAR',
        y='TOTAL_EXPENDITURE',
        color='STATE',
        title=f"Expenditure Over Time, Selected States",
    )
    return fig

if __name__=="__main__":
    app.run_server(debug=True)

In [ ]:
# example for radio buttons
app=dash.Dash(__name__)

app.layout=html.Div(
    [dcc.RadioItems(
        id='aggfunc',
        options=['mean','sum','median'],
        value='mean'
    ),
    dcc.Graph(id='graph')]
)

@app.callback(Output('graph','figure'), Input('aggfunc','value'))

def plot_rev_scatter(metric):
    fig=px.bar(
        (education.query("STATE in ['CALIFORNIA','OREGON','WASHINGTON','IDAHO'] and 1992 < YEAR < 2014")
         .groupby(['STATE'])
         .agg({'TOTAL_EXPENDITURE':f"{metric}"})
         .reset_index()),
         x='STATE',
         y='TOTAL_EXPENDITURE',
         text_auto=True,
         title=f"Expenditure Over Time, Selected States"
    )
    return fig

if __name__=='__main__':
    app.run_server(debug=True)

In [ ]:
# example for sliders
app= dash.Dash(__name__)

app.layout=html.Div(
    [
        dcc.Slider(
            id='score_slider',
            min=education['AVG_MATH_8_SCORE'].min(),
            max=education['AVG_MATH_8_SCORE'].max(),
            step=10,
            value=education['AVG_MATH_8_SCORE'].mean()
        ),
        dcc.Graph(id='graph')
    ]
)

@app.callback(Output('graph','figure'),Input('score_slider','value'))

def plot_rev_scatter(score):
    df=(
        education.query("YEAR==2013")
        .assign(score_cutoff=lambda x: np.where(
            x["AVG_MATH_8_SCORE"]>score,
            'Above Threshold',
            'Below Threshold'
        ))
    )
    fig=px.scatter(
        df,
        x='AVG_MATH_8_SCORE',
        y='expenditure_per_student',
        color='score_cutoff',
        hover_name='STATE',
        title="Score and Expenditure vs Threshold"
    )
    return fig


if __name__=='__main__':
    app.run_server(debug=True)

In [7]:
# example for range slider

app=dash.Dash(__name__)

app.layout=html.Div(
    [
        dcc.RangeSlider(
            id='score_slider',
            min=education.query('YEAR==2013')['AVG_MATH_8_SCORE'].min(),
            max=education.query('YEAR==2013')['AVG_MATH_8_SCORE'].max(),
            step=5,
            value=[270,280]
        ),
        dcc.Graph(id='graph')
    ]
)

@app.callback(Output('graph','figure'),Input('score_slider','value'))

def plot_rev_scatter(score):
    df=(
        education.query('YEAR==2013')
        .assign(score_cutoff=lambda x:np.where(
            x['AVG_MATH_8_SCORE'].between(score[0],score[1]),
            'In Range',
            'Out of Range'
        ))
    )
    fig=px.scatter(
        df,
        x='AVG_MATH_8_SCORE',
        y='expenditure_per_student',
        color='score_cutoff',
        hover_name='STATE',
        title='Score and expenditure vs Threshold'
    )
    return fig

if __name__=='__main__':
    app.run_server(debug=True)
    
